# Development grounds for code

In [15]:
from vehicle_detector import *
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from keras.models import Sequential
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, SpatialDropout2D, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

import cv2

%matplotlib inline

In [16]:
def get_data(sample_size=18458):    
    notcar_data_folders = ['./data/non-vehicles/Extras',
                           './data/non-vehicles/GTI']

    car_data_folders    = ['./data/vehicles/GTI_MiddleClose',
                            './data/vehicles/GTI_Far',
                            './data/vehicles/KITTI_extracted',
                            './data/vehicles/GTI_Right',
                            './data/vehicles/GTI_Left']

    cars = []
    notcars = []
    y_one_hot = []
    for folder in notcar_data_folders:
        image_paths =glob.glob(folder+'/*')
        for path in image_paths:
            notcars.append(cv2.imread(path))
            # One_hot_label encoding
            y_one_hot.append([0,1])
            if len(notcars) == sample_size:
                break
        if len(notcars) == sample_size:
                break


    for folder in car_data_folders:
        image_paths =glob.glob(folder+'/*')
        for path in image_paths:
            cars.append(cv2.imread(path))
            y_one_hot.append([1,0])
            # One_hot_label encoding
            if len(cars) == sample_size:
                break
        if len(cars) == sample_size:
                break


    cars = np.array(cars)
    notcars = np.array(notcars)
    y_one_hot = np.array(y_one_hot)

    X = np.concatenate((cars, notcars), axis=0)

    del cars
    del notcars
    
    with open('./data/data.p', 'wb') as f:
        pickle.dump((X, y_one_hot), f, pickle.HIGHEST_PROTOCOL)

# Uncomment for getting new samples sizes        
get_data(sample_size=10000)

with open('./data/data.p', 'rb') as f:
    X, y_one_hot = pickle.load(f)
print('loaded..')


loaded..


In [17]:
def resize_to_target_size(image):
    TARGET_SIZE = (32,32)
    return cv2.resize(image, TARGET_SIZE)

def preprocess_image(image):
    image = resize_to_target_size(image)
    image = image.astype(np.float32)
    # Normalize image
    image = image / 255.0 - 0.5
    return image

X_normalized = []

for img in X:
    X_normalized.append(preprocess_image(img))
    
X_normalized = np.array(X_normalized)

X_train, X_test, y_train, y_test = train_test_split(X_normalized,
                                                    y_one_hot,
                                                    random_state=42,
                                                    test_size=0.1)


In [19]:
model = Sequential()

model.add(Convolution2D(24, 3, 3,
                        border_mode='valid',
                        input_shape=(32,32, 3)))
model.add(Activation('relu'))
model.add(Convolution2D(48, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))



# TODO: Compile and train the model
model.compile('adam', 'binary_crossentropy', ['accuracy'])
model.fit(X_train,
          y_train,
          batch_size=64,
          nb_epoch=5,
          verbose=1,
          validation_split=0.2,
          shuffle= True)


Train on 13289 samples, validate on 3323 samples
Epoch 1/3
13289/13289 [==============================] - 113s - loss: 0.3272 - acc: 0.8592 - val_loss: 0.2225 - val_acc: 0.9151
Epoch 2/3
13289/13289 [==============================] - 97s - loss: 0.2079 - acc: 0.9248 - val_loss: 0.1879 - val_acc: 0.9335
Epoch 3/3
13289/13289 [==============================] - 94s - loss: 0.1801 - acc: 0.9369 - val_loss: 0.1640 - val_acc: 0.9404


In [20]:
model.save('./models/keras(32x32).h5')
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_11 (Convolution2D) (None, 30, 30, 24)    672         convolution2d_input_5[0][0]      
____________________________________________________________________________________________________
activation_7 (Activation)        (None, 30, 30, 24)    0           convolution2d_11[0][0]           
____________________________________________________________________________________________________
convolution2d_12 (Convolution2D) (None, 28, 28, 48)    10416       activation_7[0][0]               
____________________________________________________________________________________________________
activation_8 (Activation)        (None, 28, 28, 48)    0           convolution2d_12[0][0]           
___________________________________________________________________________________________

In [21]:
# TODO: Evaluate model on test data

metrics = model.evaluate(X_test, y_test)
for metric_i in range(len(model.metrics_names)):
    metric_name = model.metrics_names[metric_i]
    metric_value = metrics[metric_i]
    print('{}: {}'.format(metric_name, metric_value))

1846/1846 [==============================] - 4s     
loss: 0.18173784388881076
acc: 0.9355362950141096


In [ ]:
def get_model():
    """
    NVIDIA Model...
    """

    model = Sequential([

        # Conv 5x5
        Convolution2D(24, 5, 5, border_mode='same', activation='elu',
                      input_shape=(32, 32, 3)),
        MaxPooling2D(border_mode='same'),
        # Conv 5x5
        Convolution2D(36, 3, 3, border_mode='same', activation='elu'),
        MaxPooling2D(border_mode='same'),
        SpatialDropout2D(0.2),
        # Conv 3x3
        Convolution2D(48, 3, 3, border_mode='same', activation='elu'),
        MaxPooling2D(border_mode='same'),
        SpatialDropout2D(0.2),
        # Conv 3x3
        Convolution2D(64, 3, 3, border_mode='same', activation='elu'),
        MaxPooling2D(border_mode='same'),
        SpatialDropout2D(0.2),
        # Conv 3x3
        Convolution2D(64, 3, 3, border_mode='same', activation='elu'),
        MaxPooling2D(border_mode='same'),
        SpatialDropout2D(0.2),
        # Flatten
        Flatten(),
        # Fully Connected
        Dense(100, activation='elu'),
        Dense(50, activation='elu'),
        Dense(10, activation='elu'),
        Dense(2, activation='linear')
    ])
    model.add(Activation('softmax'))
    return model